In [1]:
import os
os.listdir()

['Chest-CT-Scan-data.zip', 'data_ingestion.ipynb', 'trials.ipynb', 'logs']

In [2]:
os.chdir("../")
%pwd

'/Users/ajaydesai/ajay.test/ChestCancerClassifier-MLflow-DVC-AWS'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from ChestCancerClassifier.constants import *
from ChestCancerClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [6]:
import os
import zipfile
import gdown
from ChestCancerClassifier import logger
from ChestCancerClassifier.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-24 16:39:15,425: INFO: common: yaml file: /Users/ajaydesai/ajay.test/ChestCancerClassifier-MLflow-DVC-AWS/config/config.yaml loaded successfully]
[2025-05-24 16:39:15,427: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-24 16:39:15,427: INFO: common: created directory at: artifacts]
[2025-05-24 16:39:15,428: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-24 16:39:15,428: INFO: 1983665874: Downloading data from https://drive.google.com/file/d/18sT_Q_S05ouygVTGRPtdo6Ixy_n-CEEl/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=18sT_Q_S05ouygVTGRPtdo6Ixy_n-CEEl
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=18sT_Q_S05ouygVTGRPtdo6Ixy_n-CEEl&confirm=t&uuid=d49a3a07-0cc3-45c9-aef5-ccf492d24c4c
To: /Users/ajaydesai/ajay.test/ChestCancerClassifier-MLflow-DVC-AWS/artifacts/data_ingestion/data.zip
100%|██████████| 124M/124M [00:30<00:00, 4.11MB/s] 

[2025-05-24 16:39:50,013: INFO: 1983665874: Downloaded data from https://drive.google.com/file/d/18sT_Q_S05ouygVTGRPtdo6Ixy_n-CEEl/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
